In [1]:
from premise import *
from datapackage import Package
import bw2data

In [2]:
bw2data.projects.set_current("ei39")

In [3]:
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2040},
    #{"model": "remind", "pathway":"SSP2-Base", "year": 2025},
]


fp = "/Users/romain/GitHub/RTE_scenarios/datapackage.json"

#args = {"range time":2, "duration":False, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00}

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent 3.9.1 cutoff",
        source_version="3.9.1",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[
            Package(fp)
        ],
)
ndb.update_all()

premise v.(1, 8, 2, 'dev7')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

Scenario no.:  2



//////////////////// EXTRACTING SOURCE DATABASE ////////////////////
Done!

////////////////// IMPORTING DEFAULT INVENTORIES ///////////////////
Done!

/////////////////////// EXTRACTING IAM DATA ////////////////////////
Done!
`update_all()` will skip the following steps:
update_two_wheelers(), update_cars(), and update_buses()
If you want to update these steps, please run them separately afterwards.

//////////////// IMPORTING USER-DEFINED INVENTORIES ////////////////
Extracted 1 worksheets in 0.01 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


/Users/romain/anaconda3/envs/premise/lib/python3.10/site-packages/bw2io/importers/base_lci.py:431: UserWarning: Skipping migration migration_391_38 because it isn't installed.
  warnings.warn(
/Users/romain/anaconda3/envs/premise/lib/python3.10/site-packages/bw2io/importers/base_lci.py:431: UserWarning: Skipping migration migration_391_39 because it isn't installed.
  warnings.warn(


Create custom markets.
Log file of exchanges saved under /Users/romain/Github/premise/premise/data/logs.
Done!



In [4]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway.
Running all checks...


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Title: Writing activities to SQLite3 database:
  Started: 01/29/2024 11:48:29
  Finished: 01/29/2024 11:48:49
  Total time elapsed: 00:00:20
  CPU %: 98.60
  Memory %: 9.83
Created database: ecoinvent_cutoff_3.9_image_SSP2-Base_2040_futurs-energetiques-2050
Generate scenario report.
Report saved under /Users/romain/Github/RTE_scenarios/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/Github/RTE_scenarios/dev.


In [1]:
import pandas as pd
import xarray as xr
import csv
 
def get_delimiter(data=None, filepath=None):
    sniffer = csv.Sniffer()
    if filepath:
        data = open(filepath, "r").readline()
    delimiter = sniffer.sniff(data).delimiter
    return delimiter
 
 
filename = "GAINS_emission_factors_EU.csv"
filepath = r"/Users/romain/Documents/GAINS_emission_factors_EU.csv"
 
gains_emi_EU = pd.read_csv(
    filepath,
    delimiter=get_delimiter(filepath=filepath),
    low_memory=False,
    dtype={
        "Region": str,
        "Sector": str,
        "Activity": str,
        "variable": str,
        "value": float,
        "year": int,
        "substance": str,
        "Activity_long": str,
    },
    encoding="utf-8",
)
gains_emi_EU["sector"] = gains_emi_EU["Sector"] + gains_emi_EU["Activity"]
gains_emi_EU.drop(
    [
        "Sector",
        "Activity",
    ],
    axis=1,
)
 
gains_emi_EU = gains_emi_EU[~gains_emi_EU["value"].isna()]
 
gains_emi_EU = gains_emi_EU.rename(
    columns={"Region": "region", "substance": "pollutant"}
)
 
array = (
    gains_emi_EU.groupby(["region", "pollutant", "year", "sector"])["value"]
    .mean()
    .to_xarray()
)
array = array.interpolate_na(dim="year", method="nearest", fill_value="extrapolate")
array = array.bfill(dim="year")
array = array.ffill(dim="year")

In [9]:
get_delimiter(filepath=filepath)

','

In [8]:
pd.__version__

'1.5.3'

In [2]:
gains_emi_EU

,region,Sector,Activity,value,year,pollutant,sector
0,AL,CON_COMB,GAS,0.019,2005,CO,CON_COMBGAS
1,AL,CON_COMB,HF,0.040,2005,CO,CON_COMBHF
2,AL,CON_COMB,LPG,0.015,2005,CO,CON_COMBLPG
3,AL,DOM,HF,0.043,2005,CO,DOMHF
4,AL,DOM,LPG,0.040,2005,CO,DOMLPG
...,...,...,...,...,...,...,...
288039,XK,PP_NEW_L,BC1,0.034,2050,PM10,PP_NEW_LBC1
288040,XK,PP_NEW,HF,0.009,2050,PM10,PP_NEWHF
288041,XK,IN_BO_OTH_L,BC1,0.163,2050,PM10,IN_BO_OTH_LBC1
288042,XK,IN_BO_PAP,BC1,0.160,2050,PM10,IN_BO_PAPBC1


In [4]:
array

<xarray.DataArray 'value' (region: 45, pollutant: 10, year: 6, sector: 441)>
array([[[[  nan,   nan, 8.75 , ...,   nan, 0.318,   nan],
         [  nan,   nan, 8.75 , ...,   nan, 0.318,   nan],
         [  nan,   nan, 8.75 , ...,   nan, 0.318,   nan],
         [  nan,   nan, 8.75 , ...,   nan, 0.318,   nan],
         [  nan,   nan, 8.75 , ...,   nan, 0.318,   nan],
         [  nan,   nan, 8.75 , ...,   nan, 0.318,   nan]],

        [[  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan]],

        [[  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan]],
...
        [[0.188, 0.053,   nan, ...,   nan,   nan,   nan],
         [0.188, 0.053,   nan, ...,   nan,   nan,   nan],
         [0.188, 0.053,   nan, ...,   nan,   nan,   nan],
         [0.188, 0.053,   nan, ...,   nan,   nan,   nan],
         [0.188, 0.053,   nan, ...,   nan,   nan,   nan],
         [0.188, 0.053,   nan, ...,   nan,   nan,   nan]],

        [[  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan]],

        [[  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan],
         [  nan,   nan,   nan, ...,   nan,   nan,   nan]]]])
Coordinates:
  * region     (region) object 'AL' 'AM' 'AT' 'AZ' 'BA' ... 'SK' 'TR' 'UA' 'XK'
  * pollutant  (pollutant) object 'CH4' 'CO' 'N2O' 'NH3' ... 'PM25' 'SO2' 'VOC'
  * year       (year) int64 2005 2010 2020 2030 2040 2050
  * sector     (sector) object 'AGR_ARABLENOF' ... 'WW_DOM_DC_TRMPOP'

In [4]:
ndb.write_db_to_matrices()

Write new database(s) to matrix.
Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
3.9
Matrices saved in /Users/romain/GitHub/RTE_scenarios/dev/export/image/SSP2-Base/2020.
Generate scenario report.
Report saved under /Users/romain/GitHub/RTE_scenarios/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/RTE_scenarios/dev.


In [4]:
ndb.write_db_to_brightway(
    [
        "RTE_2020_M0",
        #"RTE_2025_sobriety_N3",
        #"RTE_2030_sobriety_N3",
        #"RTE_2035_sobriety_N3",
        #"RTE_2040_sobriety_N3",
        #"RTE_2045_sobriety_N3",
        #"RTE_2050_sobriety_N3",
        #"RTE_2055_sobriety_N3",
        #"RTE_2060_sobriety_N3"
    ]
)

Write new database(s) to Brightway2.
Prepare database 1.
- check for duplicates...
- check for values format...
One or multiple exchanges have a wrong format.
+---------------------------------------------------------------------------------+----------+------------+-------+--------------------+
|                                       Name                                      | Location |   Field    | Value |        Type        |
+---------------------------------------------------------------------------------+----------+------------+-------+--------------------+
| hydrogen, recovered from coke oven gas, with carbon capture and storage, FE2050 |    FR    | categories |  None | <class 'NoneType'> |
+---------------------------------------------------------------------------------+----------+------------+-------+--------------------+
This may create issues later on.
- relinking exchanges...
Done!
Database RTE_2020_M0 already exists: it will be overwritten.
Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:31


Title: Writing activities to SQLite3 database:
  Started: 03/29/2023 16:05:11
  Finished: 03/29/2023 16:05:43
  Total time elapsed: 00:00:31
  CPU %: 96.90
  Memory %: 10.83
Created database: RTE_2020_M0
Generate scenario report.
Report saved under /Users/romain/GitHub/RTE_scenarios/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/RTE_scenarios/dev.


In [5]:
dbs = [x for x in bw.databases if "RTE" in x]
FU = []

for db in dbs:
    for act in bw.Database(db):
        if "FE2050" in act["name"] and "hydrogen" in act["name"]:
            FU.append({act: 1})
            

In [8]:
import bw2calc
res = []
method = ('IPCC 2013 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')
for fu in FU:
    lca = bw2calc.LCA(fu, method)
    lca.lci()
    lca.lcia()
    res.append([list(fu.keys())[0]["name"], list(fu.keys())[0]["database"], lca.score])

/opt/homebrew/Caskroom/miniforge/base/envs/ab/lib/python3.9/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.39e+14)
  warnings.warn(msg, UmfpackWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/ab/lib/python3.9/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 6.16e+13)
  warnings.warn(msg, UmfpackWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/ab/lib/python3.9/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/ab/lib/python3.9/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.13e+13)
  warnings.warn(msg, UmfpackWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/ab/lib/python3.9/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning

In [9]:
import pandas as pd
pd.DataFrame(res, columns=["name", "database", "score"]).to_excel("scores.xlsx")

In [11]:
from polyviz import sankey

In [22]:
act = [a for a in bw.Database("RTE_2020_sobriety_N3") if "market for hydrogen" in a["name"] and "FE2050" in a["name"] and a["location"]=="FR"][0]
act

'market for hydrogen, gaseous, for steel use, FE2050' (kilogram, FR, None)

In [23]:
sankey(
    act,
    ('Cumulative Energy Demand (CED)', 'total', 'energy content (HHV)'),
    level=5
)

Calculating supply chain score...


/opt/homebrew/Caskroom/miniforge/base/envs/ab/lib/python3.9/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


'/Users/romain/GitHub/RTE_scenarios/dev/market for hydrogen gaseous for steel use FE2050 kilogram FR Cumulative Energy Demand CEDtotalenergy content HHV sankey.html'

In [24]:
for ds in ndb.scenarios[0]["database"]:
    for v in ds.values():
        if isinstance(v, np.ndarray):
            print(ds["name"])
    for e in ds["exchanges"]:
        for v in e.values():
            if isinstance(v, np.ndarray):
                print(ds["name"])
        

NameError: name 'ndb' is not defined

In [13]:
bw.projects.set_current("ei39")

In [14]:
list(bw.databases)

['biosphere3',
 'ecoinvent 3.9 cutoff',
 'ESA LCA External 1.1.8a',
 'ecoinvent 3.9 consequential',
 'ecoinvent_cutoff_remind_SSP1-Base_2005',
 'ecoinvent_cutoff_image_SSP2-Base_2100',
 'ecoinvent_cutoff_remind_SSP5-Base_2050',
 'ecoinvent_cutoff_3.9_remind_SSP1-Base_2005',
 'ecoinvent_cutoff_3.9_image_SSP2-Base_2100',
 'ecoinvent_cutoff_3.9_remind_SSP5-Base_2050',
 'test1',
 'test2',
 'ecoinvent 3.9.1 cutoff',
 'ecoinvent 3.9.1 consequential',
 'test_super',
 'brian',
 'RTE_2020',
 'RTE_2030',
 'RTE_2040',
 'RTE_2050',
 'RTE_2060',
 'RTE_2020_M0',
 'RTE_2025_M0',
 'RTE_2030_M0',
 'RTE_2035_M0',
 'RTE_2040_M0',
 'RTE_2045_M0',
 'RTE_2050_M0',
 'RTE_2055_M0',
 'RTE_2060_M0',
 'RTE_2020_sobriety_N3',
 'RTE_2025_sobriety_N3',
 'RTE_2030_sobriety_N3',
 'RTE_2035_sobriety_N3',
 'RTE_2040_sobriety_N3',
 'RTE_2045_sobriety_N3',
 'RTE_2050_sobriety_N3',
 'RTE_2055_sobriety_N3',
 'RTE_2060_sobriety_N3',
 'RTE_2020_ext_N1',
 'RTE_2025_ext_N1',
 'RTE_2030_ext_N1',
 'RTE_2035_ext_N1',
 'RTE_2040_e

In [11]:
import numpy as np
for ds in bw.Database("ecoinvent_remind_SSP2-PkBudg1150_2050"):
    for k, v in ds.items():
        if isinstance(v, np.ndarray):
            print(ds["name"])
        if isinstance(k, np.ndarray):
            print(ds["name"])
    for e in ds.exchanges():
        for k, v in e.items():
            if isinstance(v, np.ndarray):
                print(ds["name"])
            if isinstance(k, np.ndarray):
                print(ds["name"])

In [6]:
import numpy as np
for ds in bw.Database("RTE_2020_M0"):
    if "log parameters" in ds:
        for k, v in ds["log parameters"].items():
            if isinstance(v, np.float64):
                print(v)

In [7]:
for ds in bw.Database("RTE_2020_M0"):
    for k, v in ds.items():
        if not type(v) in [str, float, int, tuple, dict]:
            if type(v) == list:
                for x in v:
                    if not type(x) in [str, float, int, tuple, dict]:
                        print(ds["name"], k, v, x, type(x))
            if type(v) == dict:
                for i, j in v.items():
                    if not type(j) in [str, float, int, tuple, dict]:
                        print(ds["name"], k, v, i, type(i))
            else:
                print(ds["name"], type(v))
    for e in ds.exchanges():
        for k, v in e.items():
            if not type(v) in [str, float, int, tuple, dict]:
                print(ds["name"], type(v))

market for manganese dioxide <class 'list'>
market for inverter, 2.5kW <class 'list'>
market for fibre, polyester <class 'list'>
heat production, at heat pump 30kW, allocation exergy <class 'list'>
slag landfill construction <class 'list'>
slag landfill construction <class 'list'>
slag landfill construction <class 'list'>
treatment of waste glass, unsanitary landfill, wet infiltration class (500mm) <class 'list'>
market for electricity, high voltage <class 'list'>
market for waste polyethylene <class 'list'>
market for cement, CEM II/B-S <class 'list'>
butter production, from cow milk <class 'list'>
butter production, from cow milk <class 'list'>
butter production, from cow milk <class 'list'>
electricity production, wind, >3MW turbine, onshore <class 'list'>
electricity production, wind, >3MW turbine, onshore <class 'list'>
electricity production, wind, >3MW turbine, onshore <class 'list'>
market for cement, type IS <class 'list'>
market for land use change, annual crop <class 'list'>

In [6]:
bw.BW2Package.export_obj(bw.Database("RTE_2020_M0"))

TypeError: Object of type ndarray is not JSON serializable